# How to Use

1. Run everything in the **Setup** section. 
    - Make sure to change the working directory to **your** working directory. The code for this is already there.
    - Make sure the Excel document for logging the scores also exists in your working directory, and that the file name is correct.

2. Determine *if the test needs to be run* by having a good understanding of what each test is doing. 
    - Please refer to this document [here](https://086gc.sharepoint.com/:x:/r/sites/PacificSalmonTeam/Shared%20Documents/General/02%20-%20PSSI%20Secretariat%20Teams/04%20-%20Strategic%20Salmon%20Data%20Policy%20and%20Analytics/02%20-%20Data%20Governance/00%20-%20Projects/10%20-%20Data%20Quality/Presentation/DQP%20Demo.xlsx?d=wc15abe6743954df980a05f09fe99a560&csf=1&web=1&e=CJeb6h)

3. Some requirements for the datasets:
    - The data must be on the **first sheet** in the Excel document.
    - The **first row** must be the column names. 
    - The test won't run if the Excel file is open

4. After running all the tests, the Excel document for logging the scores can be uploaded to Sharepoint using the function "Saving the file to sharepoint". 

Note: The Output Reports are used for when a data steward is asking about why their dataset gets a certain score. If the metric is not in Output Reports, then running the test itself will generate an output that can be put into a report.  

# Setup

Please run everything in the set up, and double check the working directory so that the data can be read from that same directory.

All of these functions are used in the process of calculating data quality. 

In [1]:
from IPython import get_ipython

# Clear memory
get_ipython().magic('reset -sf')

C:\Users\onakd\AppData\Local\Temp\1\ipykernel_27876\1945100915.py:4: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -sf')


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os
import re
from difflib import SequenceMatcher
from datetime import datetime
import nbformat
import gc


In [3]:
gc.collect()

21

Support functions to allow running cells from other notebooks.

In [4]:
def run_selected_cells(notebook_path, cell_indices):       
    # Load the notebook       
    with open(notebook_path) as f:           
        nb = nbformat.read(f, as_version=4)   

    # Get the cells to run       
    selected_cells = [nb.cells[i] for i in cell_indices]   

    # Execute the selected cells       
    for cell in selected_cells:           
        if cell.cell_type == 'code':               
            exec(cell.source, globals())

def run_selected_cells_from_util(util_folder, notebook_name, cell_indices):       
    notebook_path = os.path.join(util_folder, notebook_name)       
    run_selected_cells(notebook_path, cell_indices)   

Make sure to set to the correct working directory

In [5]:
# Change working directory to the same place where you saved the test datasets
# os.chdir('C:/Users/luos/OneDrive - DFO-MPO/Python') #change directory
os.getcwd()  # check where the directory is (and whether the change was successful or not)
GLOBAL_USER = "OnakD"
GLOBAL_DATASET = "SEP Production Plans"
GLOBAL_DATAFILE = "2015-sep-production-plan-en (1).csv"

Function to read either csv or xlsx data 

In [30]:
# Function 0: Reading the dataset file
def read_data(dataset_path):
    _, file_extension = os.path.splitext(dataset_path)
    if file_extension == ".csv":
        try:  
            df = pd.read_csv(dataset_path, encoding="utf-8-sig")  
        except UnicodeDecodeError:  
            df = pd.read_csv(dataset_path, encoding="cp1252") 
    elif file_extension == ".xlsx":
        df = pd.read_excel(dataset_path)
    else:
        print("Unsupported file type")
        df = None
    return df

Function to log the scores into an xlsx file (already created, existing)

In [7]:
# Function to log a new row into the DQS_Log_XX.xlsx file
def log_score(test_name, dataset_name, score, selected_columns, threshold=None):
    # Convert score to a percentage
    percentage_score = score

    # Load the Excel file into a DataFrame
    log_file = "DQS_Log_Beta.xlsx"

    # Set threshold to "No threshold" if it is not provided
    if threshold is None:
        threshold_value = "no threshold"
    else:
        threshold_value = threshold

    # If selected_columns is None, assume "All" was tested
    if selected_columns is None:
        columns_tested = "All columns"
    else:
        # Convert selected_columns list to a string if specific columns are provided
        columns_tested = ", ".join(selected_columns)

    # Try loading the existing Excel file
    try:
        df = read_data(log_file)
    except FileNotFoundError:
        # Create an empty DataFrame if file doesn't exist (shouldn't be the case if you already created it)
        df = pd.DataFrame(
            columns=["Dataset", "Test", "Threshold", "Date_Calculated", "Score"]
        )

    # Prepare the new row as a DataFrame
    new_row = pd.DataFrame(
        {
            "Dataset": [dataset_name],
            "Columns_Tested": [columns_tested],  # Add the list of columns tested
            "Test": [test_name],
            "Date_Calculated": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")],
            "Threshold": [threshold_value],
            "Score": [percentage_score],
            "User": GLOBAL_USER
        }
    )

    # Append the new row to the DataFrame
    df = pd.concat([df, new_row], ignore_index=True)

    # Save the updated DataFrame back to the Excel file
    df.to_excel(log_file, index=False)

Function to extract dataset name from a path

In [8]:
def get_dataset_name(dataset_path):
    # Extract the file name from the path (e.g., 'Dataset_A.csv')
    file_name = os.path.basename(dataset_path)
    # Split the file name to remove the extension (e.g., 'Dataset_A')
    dataset_name = os.path.splitext(file_name)[0]
    return dataset_name

# Data Quality Tests

### Consistency

#### Consistency Type 1 (C1)

Calculate consistency score of a dataset

This code is best run on CSV data where the column names are in the first row. It can also accept files that are in xlsx formats but it will only take data from the first sheet if there are more than one sheet in the excel file.

Limitations: It will not check for differences in capitalization of the same word (since all the words will be changed to lower case before the similarity score is calculated)

In [28]:
# Run utils for C1 
run_selected_cells_from_util('utils', 'consistancy_utils.ipynb', [2])    

##### Test the dataset by changing the path

In [31]:
try: 
    datafilepath = f"C:/Users/{GLOBAL_USER}/OneDrive - DFO-MPO/04 - Strategic Salmon Data Policy and Analytics/07 - Data Products & Data/21 - Transitory Files/{GLOBAL_DATASET}/{GLOBAL_DATAFILE}"
    # Test Consistency Calculations

    processed_df = process_and_calculate_similarity(
        dataset_path=datafilepath,
        column_names=["Regional Area", "Project", "Project Contact", "Stock", "Stock CU Name", "Release Site/ Transfer Project", "Mark Type", "Comment"],
        threshold=0.91
    )

    # processed_df['Overall Consistency Score'].min()
    print(processed_df)
except MemoryError as e:
    print("Dataset is too large for this test, out of memory!")
    print(e)
except KeyError as e:
    print("Issue with column names, are you sure you entered them correctly?")
    print(e)
except FileNotFoundError as e:
    print("Did not find dataset, make sure you have provided the correct name.")
    print(e)


test1
                     Regional Area Program Code       Project  \
0                      BC Interior          CDP     Deadman R   
1                      BC Interior          CDP     Deadman R   
2                      BC Interior          CDP     Deadman R   
3                      BC Interior          CDP     Deadman R   
4                      BC Interior          OPS   Gates Sp Ch   
..                             ...          ...           ...   
624             Lower Fraser River          OPS  Weaver Sp Ch   
625  Yukon and Transboundary River          OPS    Snettisham   
626  Yukon and Transboundary River          OPS    Snettisham   
627  Yukon and Transboundary River          OPS    Snettisham   
628  Yukon and Transboundary River          OPS    Snettisham   

                                   Project Contact  Species     Run  \
0    CA - Central Interior, Boston Bar to 100 Mile     Coho    Fall   
1    CA - Central Interior, Boston Bar to 100 Mile     Coho    Fall   


#### Consistency Type 2 (C2)

Calculate consistency score of datasets with a reference list

The compared columns in question must be identical to the ref list, otherwise they will be penalized more harshly.

In [11]:
# Run utils for C2
run_selected_cells_from_util('utils', 'consistancy_utils.ipynb', [4]) 

##### Test the dataset by changing the path

In [12]:
# column_mapping = {
#     "STOCK_CU_NAME": "CU_Display",
#     "STOCK_CU_INDEX": "FULL_CU_IN",
# }  # the pattern for comparison is 'dataset column' : 'reference column'
# process_and_calculate_similarity_ref(
#     dataset_path="data/test/2024-03-28 1_qryThermal_NatEmerg.xlsx",
#     column_mapping=column_mapping,
#     ref_dataset_path="data/Pacific Salmon Population Unit Crosswalk_Final_20240513.xlsx",
#     threshold=1,
#     Stop_Words=[""],
# )

### Accuracy

#### Accuracy Type 1 (A1, Mixed Data Types, Symbols in Numerics) 

Test whether there are symbols in numerics

In [13]:
# Run utils for A1
run_selected_cells_from_util('utils', 'accuracy_utils.ipynb', [2]) 

##### Test the dataset by changing the path

In [14]:
try:
    datafilepath = f"C:/Users/{GLOBAL_USER}/OneDrive - DFO-MPO/04 - Strategic Salmon Data Policy and Analytics/07 - Data Products & Data/21 - Transitory Files/{GLOBAL_DATASET}/{GLOBAL_DATAFILE}"
    score = accuracy_score(
        dataset_path=datafilepath,
        selected_columns=["Egg Target ", "Release/ Transfer Target", "Target # to Mark"],
    )
    print(score)
except KeyError as e:
    print("Issue with column names, are you sure you entered them correctly?")
    print(e)
except Exception as e:
    print("Test failed!")
    print(e)

None


#### Accuracy Type 2 (A2 Outliers)

In [15]:
# Run utils for A2
run_selected_cells_from_util('utils', 'accuracy_utils.ipynb', [4]) 

Tests

In [16]:
try:    
    datafilepath = f"C:/Users/{GLOBAL_USER}/OneDrive - DFO-MPO/04 - Strategic Salmon Data Policy and Analytics/07 - Data Products & Data/21 - Transitory Files/{GLOBAL_DATASET}/{GLOBAL_DATAFILE}"
    outliers = find_outliers_iqr(
        dataset_path=datafilepath,
        selected_columns = ["Egg Target ", "Release/ Transfer Target", "Target # to Mark"],
        threshold=0.90,
        minimum_score=0.85,
    )
    print(outliers)
except KeyError as e:
    print("Issue with column names, are you sure you entered them correctly?")
    print(e)
except Exception as e:
    print("Test failed!")
    print(e)

Issue with column names, are you sure you entered them correctly?
'Egg Target '


#### Accuracy Type 3 (A3 Duplicates)

In [17]:
# Run utils for A3
run_selected_cells_from_util('utils', 'accuracy_utils.ipynb', [6]) 

Test

In [18]:
try:
    datafilepath = f"C:/Users/{GLOBAL_USER}/OneDrive - DFO-MPO/04 - Strategic Salmon Data Policy and Analytics/07 - Data Products & Data/21 - Transitory Files/{GLOBAL_DATASET}/{GLOBAL_DATAFILE}"
    find_duplicates_and_percentage(
        dataset_path=datafilepath
    )

except Exception as e:
    print("Test failed!")
    print(e)

Duplicate Rows:
Empty DataFrame
Columns: [ï»¿Regional Area, Program Code, Project, Project Contact, Species, Run, Stock, Stock CU Index, Stock CU Name, Stock Stat Area, Objective, Production Strategy Level,  Egg Target , Production Activity Type, Release/ Transfer Stage, Release Site/ Transfer Project, Release Stat Area,  Release/ Transfer Target , Mark Type, Clip Type,  Target # to Mark , Comment]
Index: []

[0 rows x 22 columns]

Duplication Score: 100.0%


### Completeness (P)

The threshold is for removing a column that meets the threshold of the percentage of blanks.

In [19]:
def completeness_test(dataset_path, exclude_columns=[], threshold=0.75):
    dataset = read_data(dataset_path)

    # Exclude the 'Comment' column if it exists in the dataset
    if "Comment" in dataset.columns:
        dataset = dataset.drop(columns=["Comment"])

    # Exclude columns in exclude_columns if they exist in the dataset
    dataset = dataset.drop(
        columns=[col for col in exclude_columns if col in dataset.columns]
    )

    # Calculate the percentage of non-null (non-missing) values in each column
    is_null_percentage = dataset.isna().mean()

    # Identify columns with non-null percentage less than or equal to the threshold
    columns_to_keep = is_null_percentage[is_null_percentage <= threshold].index

    # Keep columns that exceed the threshold of non-null values
    dataset2 = dataset[columns_to_keep]

    # Calculate the actual percentage of non-missing values in the dataset
    total_non_missing = dataset2.notna().sum().sum()
    total_obs = dataset2.shape[0] * dataset2.shape[1]
    completeness_score = total_non_missing / total_obs

    # log the results
    log_score(
        test_name="Completeness (P)",
        dataset_name=get_dataset_name(dataset_path),
        selected_columns=None,
        threshold=threshold,
        score=completeness_score,
    )

    return completeness_score

Test

In [20]:
# "North and Central Coast NuSEDS_20241004.xlsx"
# "West Coast Vancouver Island NuSEDS_20241004.xlsx"
# "Yukon and Transboundary NuSEDS_20241004.xlsx"
try:
    datafilepath = f"C:/Users/{GLOBAL_USER}/OneDrive - DFO-MPO/04 - Strategic Salmon Data Policy and Analytics/07 - Data Products & Data/21 - Transitory Files/{GLOBAL_DATASET}/{GLOBAL_DATAFILE}"
    completeness_test(
        datafilepath,
        threshold=0.75,
    )
except Exception as e:
    print("Test failed!")
    print(e)

### Timeliness

In [21]:
from datetime import datetime


def calc_timeliness(refresh_date, cycle_day):
    refresh_date = pd.to_datetime(refresh_date)
    unupdate_cycle = np.max([((datetime.now() - refresh_date).days / cycle_day) - 1, 0])

    # unupdate_cycle = np.floor((datetime.now() - refresh_date).days/cycle_day)
    # print((datetime.now() - refresh_date).days/cycle_day)
    return np.max([0, 100 - (unupdate_cycle * (100 / 3))])

In [22]:
calc_timeliness("2022-12-01", cycle_day=365)

66.30136986301369

# Output Reports
Run all the functions above first before running this section

#### Note that output reports can be generated through the data quality tests of
<p>    - Consistency type 1
<p>    - Accuracy type 2
<p>    - Accuracy type 3
<p>    - Completeness
<p>          
<p>  *Completeness test does not require an output report (just find the blanks in the dataset). The rest can be found below

### Consistency Type 2

In [23]:
def compare_datasets(dataset_path, column_mapping, ref_dataset_path=None):
    # Read the data file
    df = read_data(dataset_path)

    # Initialize ref_df if a ref dataset is provided
    if ref_dataset_path:
        df_ref = read_data(ref_dataset_path)
        ref_data = True  # Flag to indicate we are using a ref dataset
    else:
        ref_data = False  # No ref dataset, compare within the same dataset

    for selected_column, m_selected_column in column_mapping.items():
        if ref_data:
            # Compare to ref dataset
            unique_observations = get_names_used_for_column(df_ref, m_selected_column)
        else:
            # Use own column for comparison
            unique_observations = get_names_used_for_column(df, selected_column)

        # Iterate over each row in the selected column
        column_results = []
        for value in df[selected_column]:
            # Check if the value exists in unique_observations and append the result to column_results
            if pd.isnull(value):
                column_results.append(
                    False
                )  # or True, depending on how you want to handle NaN values
            else:
                column_results.append(value in unique_observations)

        # Add the results as a new column in the DataFrame
        df[selected_column + "_comparison"] = column_results

    return df

In [24]:
column_mapping = {
    "STOCK_CU_NAME": "CU_Display",
    "STOCK_CU_INDEX": "FULL_CU_IN",
}  # the pattern for comparison is 'dataset column' : 'reference column'
compare_datasets(
    dataset_path="data/test/Salmonid_Enhancement_Program_Releases.xlsx",
    column_mapping=column_mapping,
    ref_dataset_path="data/Pacific Salmon Population Unit Crosswalk_Final_20240513.xlsx",
)

FileNotFoundError: [Errno 2] No such file or directory: 'data/test/Salmonid_Enhancement_Program_Releases.xlsx'

### Accuracy Type 1

In [ ]:
# Function 1: Using isdigit to find non-numerical entries
def find_non_digits(s):
    # Ensure the value is treated as a string
    s = str(s)
    return [char for char in s if not (char.isdigit() or char == ".")]


# Function 2 : Check if each row has only numbers in each selected column and add results as new columns
def add_only_numbers_columns(dataset_path, selected_columns):
    adf = read_data(dataset_path)
    selected_columns = [col for col in adf.columns if col in selected_columns]

    for column_name in selected_columns:
        adf[column_name + "_Only_Numbers"] = adf[column_name].apply(
            lambda x: len(find_non_digits(x)) == 0
        )

    return adf

Test

In [ ]:
add_only_numbers_columns(
    dataset_path="data/test/SEP Facilities.xlsx", selected_columns=["LicNo", "FRN"]
)

# Score Log

In [ ]:
from datetime import datetime

current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

current_date

'2024-11-28 16:53:40'

In [ ]:
def get_dataset_name(dataset_path):
    # Extract the file name from the path (e.g., 'Dataset_A.csv')
    file_name = os.path.basename(dataset_path)
    # Split the file name to remove the extension (e.g., 'Dataset_A')
    dataset_name = os.path.splitext(file_name)[0]
    return dataset_name

More up to date code for the score log can be found in the "Setup" section, the code here is treated more as a testing space

In [ ]:
# Function to log a new row into the DQS_Log.xlsx file
def log_score(test_name, dataset_name, score, threshold=None):
    # Convert score to a percentage
    percentage_score = score * 100

    # Load the Excel file into a DataFrame
    log_file = "DQS_Log_Test.xlsx"

    # Set threshold to "No threshold" if it is not provided
    if threshold is None:
        threshold_value = "no threshold"
    else:
        threshold_value = threshold
    # Try loading the existing Excel file
    try:
        df = read_data(log_file)
    except FileNotFoundError:
        # Create an empty DataFrame if file doesn't exist (shouldn't be the case if you already created it)
        df = pd.DataFrame(
            columns=["Dataset", "Test", "Threshold", "Date_Calculated", "Score"]
        )

    # Prepare the new row as a DataFrame
    new_row = pd.DataFrame(
        {
            "Dataset": [dataset_name],
            "Test": [test_name],
            "Date_Calculated": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")],
            "Threshold": [threshold_value],
            "Score": [percentage_score],
        }
    )

    # Append the new row to the DataFrame
    df = pd.concat([df, new_row], ignore_index=True)

    # Save the updated DataFrame back to the Excel file
    df.to_excel(log_file, index=False)

### Saving the file to sharepoint

In [ ]:
import shutil
import os


# Function to copy the log file to another folder
def copy_log_file(destination_folder):
    # Define the name of the file and the current working directory
    log_file = "DQS_Log_Test.xlsx"

    # Get the current working directory (if needed)
    current_directory = os.getcwd()

    # Define the source path (current working directory + file)
    source_path = os.path.join(current_directory, log_file)

    # Define the destination path (destination folder + file)
    destination_path = os.path.join(destination_folder, log_file)

    # Copy the file to the destination folder
    shutil.copy(source_path, destination_path)

    print(f"File copied to {destination_path}")

Run this function when saving the excel document from the working directory to Sharepoint:

In [ ]:
# Specify the destination folder where you want to copy the file
destination_folder = "C:/Users/EwertM/Documents/Portal/DataQuality"

copy_log_file(destination_folder)